<a href="https://colab.research.google.com/github/cliverenwood/audioposter/blob/main/Audio_Poster_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y ffmpeg
!pip install yt-dlp openai-whisper sentence-transformers umap-learn plotly qrcode[pil] matplotlib pandas
!pip install librosa

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=a52821dd32c0c0e420235603e16b2dbd08f70a872d0596dfc1ffd5826e896ace
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [ ]:
import yt_dlp
import whisper
import torch
from sentence_transformers import SentenceTransformer
import umap
import pandas as pd
import plotly.express as px
import qrcode
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import io
import numpy as np

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# --- USER INPUT ---
YOUTUBE_URL = "https://www.youtube.com/watch?v=9fyH9M3l2pw" # Replace with your URL
# ------------------

OUTPUT_AUDIO = "audio.mp3"
MODEL_WHISPER = "base" # Options: tiny, base, small, medium, large (large is slower but more accurate)
MODEL_EMBEDDING = "all-MiniLM-L6-v2" # Fast and effective for clustering

In [ ]:
print("--- 1. Downloading Audio ---")
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'mp3','preferredquality': '192'}],
    'outtmpl': 'audio'
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([YOUTUBE_URL])

print("--- 2. Transcribing Audio (This may take a moment) ---")
# Load the model (using "base" or "small" is good for CPU)
model = whisper.load_model(MODEL_WHISPER, device=device)

# FIX: Use OUTPUT_AUDIO directly. Do not add + ".mp3"
result = model.transcribe(OUTPUT_AUDIO)

# Extract segments
segments = result['segments']
texts = [seg['text'].strip() for seg in segments]
timestamps = [seg['start'] for seg in segments]

df = pd.DataFrame({'text': texts, 'start_time': timestamps})
print(f"Transcription complete. Extracted {len(df)} segments.")

--- 1. Downloading Audio ---
[youtube] Extracting URL: https://www.youtube.com/watch?v=9fyH9M3l2pw
[youtube] 9fyH9M3l2pw: Downloading webpage


[youtube] 9fyH9M3l2pw: Downloading android sdkless player API JSON
[youtube] 9fyH9M3l2pw: Downloading web safari player API JSON


[youtube] 9fyH9M3l2pw: Downloading m3u8 information


[info] 9fyH9M3l2pw: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of    5.81MiB in 00:00:00 at 19.61MiB/s  
[ExtractAudio] Destination: audio.mp3
Deleting original file audio (pass -k to keep)
--- 2. Transcribing Audio (This may take a moment) ---


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 95.8MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription complete. Extracted 35 segments.


In [ ]:
print("--- 3. Generating Embeddings ---")
embedder = SentenceTransformer(MODEL_EMBEDDING)
embeddings = embedder.encode(texts, show_progress_bar=True)

print("--- 4. Reducing Dimensions to 3D with UMAP ---")
# Adjust n_neighbors: larger = more global structure, smaller = more local clustering
reducer = umap.UMAP(n_components=3, n_neighbors=15, min_dist=0.1, random_state=42)
projections = reducer.fit_transform(embeddings)

df['x'] = projections[:, 0]
df['y'] = projections[:, 1]
df['z'] = projections[:, 2]

print("Data processing complete.")

--- 3. Generating Embeddings ---


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

--- 4. Reducing Dimensions to 3D with UMAP ---


/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Data processing complete.


In [ ]:
fig = px.scatter_3d(
    df, x='x', y='y', z='z',
    hover_data=['text'],
    color='z',
    color_continuous_scale='Viridis',
    title="Semantic Cluster Map (Interactive)",
    template="plotly_dark"
)

# Enhanced photorealistic rendering settings
fig.update_traces(
    marker=dict(
        size=6,
        opacity=0.85,
        line=dict(width=0.5, color='rgba(255, 255, 255, 0.3)'),  # Rim lighting effect
        # Add gradient-like shading
    ),
    # Add subtle glow effect
)

# Photorealistic camera and lighting
fig.update_layout(
    scene=dict(
        xaxis=dict(
            backgroundcolor="rgb(20, 20, 30)",
            gridcolor="rgba(255, 255, 255, 0.1)",
            showbackground=True,
            zerolinecolor="rgba(255, 255, 255, 0.2)",
        ),
        yaxis=dict(
            backgroundcolor="rgb(20, 20, 30)",
            gridcolor="rgba(255, 255, 255, 0.1)",
            showbackground=True,
            zerolinecolor="rgba(255, 255, 255, 0.2)",
        ),
        zaxis=dict(
            backgroundcolor="rgb(20, 20, 30)",
            gridcolor="rgba(255, 255, 255, 0.1)",
            showbackground=True,
            zerolinecolor="rgba(255, 255, 255, 0.2)",
        ),
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.3),  # Cinematic angle
            center=dict(x=0, y=0, z=0)
        ),
        # Enhanced lighting for depth perception
        aspectmode='cube'
    ),
    paper_bgcolor='rgb(10, 10, 15)',
    plot_bgcolor='rgb(10, 10, 15)',
)

fig.show()


In [ ]:
def project_3d_to_2d(x, y, z, W, H, scale, rot_y, rot_x, z_scale):
    """Projects a 3D point to a 2D canvas with rotation and perspective."""
    # Rotate around X axis
    rx = np.radians(rot_x)
    ry = np.radians(rot_y)
    cz = np.cos(rx)
    sz = np.sin(rx)
    ny = y * cz - z * sz
    nz = y * sz + z * cz
    y, z = ny, nz

    # Rotate around Y axis
    cy = np.cos(ry)
    sy = np.sin(ry)
    nx = x * cy - z * sy
    nz = x * sy + z * cy
    x, z = nx, nz

    # Apply perspective projection
    # z_scale controls how strong the perspective is (larger z_scale means less perspective distortion)
    perspective_factor = 1 / (1 + z / z_scale)

    px = W / 2 + x * scale * perspective_factor
    py = H / 2 - y * scale * perspective_factor # Y-axis inverted for PIL

    return px, py, z # Return z for sorting nodes by depth

In [ ]:
import subprocess
import sys
import os

# --- 0. INSTALLATION & IMPORTS (ROBUST) ---
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import qrcode
except ImportError:
    print("Installing qrcode...")
    install("qrcode[pil]")
    import qrcode

try:
    import librosa
except ImportError:
    print("Installing librosa...")
    install("librosa")
    import librosa

try:
    import yt_dlp
except ImportError:
    print("Installing yt-dlp...")
    install("yt-dlp")
    import yt_dlp

import numpy as np
import pandas as pd
import math
from PIL import Image, ImageDraw, ImageFont, ImageFilter

# --- 1. KONFIGURATION ---
OUTPUT_AUDIO = "audio.mp3"
YOUTUBE_URL = "https://www.youtube.com/watch?v=9fyH9M3l2pw"

# Farben (Neon Vision Palette)
GRADIENT_COLORS = [
    (21,143,191),   # Violett
    (242,203,12),   # Grün
    (242,161,12),   # Gelb
    (242,119,12),   # Orange
    (242,12,12)    # Rot
]

def get_gradient_color(norm_value):
    val = norm_value * 4
    val = max(0, min(val, 4))
    idx1 = int(val)
    idx2 = min(idx1 + 1, 4)
    t = val - idx1
    c1 = GRADIENT_COLORS[idx1]
    c2 = GRADIENT_COLORS[idx2]
    r = int(c1[0] + (c2[0] - c1[0]) * t)
    g = int(c1[1] + (c2[1] - c1[1]) * t)
    b = int(c1[2] + (c2[2] - c1[2]) * t)
    return (r, g, b)

# --- 2. DATENVERARBEITUNG ---
print("--- Processing Audio Data ---")

try:
    y, sr = librosa.load(OUTPUT_AUDIO, sr=None)
except FileNotFoundError:
    print("WARNUNG: 'audio.mp3' nicht gefunden. Generiere Dummy-Daten.")
    y = np.random.uniform(-0.5, 0.5, 22050*30)
    sr = 22050

duration = librosa.get_duration(y=y, sr=sr)
n_chunks = 150
chunk_duration = duration / n_chunks

rms_values = []
for i in range(n_chunks):
    start = int(i * chunk_duration * sr)
    end = int((i + 1) * chunk_duration * sr)
    chunk = y[start:end]
    rms = np.mean(librosa.feature.rms(y=chunk)) if len(chunk) > 0 else 0
    rms_values.append(rms)

rms_max = np.max(rms_values) if np.max(rms_values) > 0 else 1
z_vals = np.array(rms_values) / rms_max

x_vals = []
y_vals = []
golden_angle = 137.508 * (math.pi / 180.0)
spread = 0.8

for i in range(n_chunks):
    r = spread * (i / n_chunks)**0.5
    theta = i * golden_angle
    x_vals.append(r * math.cos(theta))
    y_vals.append(r * math.sin(theta))

df_final = pd.DataFrame({
    'x': x_vals,
    'y': y_vals,
    'RMS_Z': z_vals,
    'color': [get_gradient_color(z) for z in z_vals]
})
df_final['z_plot'] = df_final['RMS_Z'] - 0.5


# --- 3. VERBESSERTER 3D RENDERER ---
def project_3d_to_2d(x, y, z, width, height, scale, rotation_y=0, rotation_x=0, z_scale=1.0):
    """Verbesserte 3D-Projektion mit Z-Skalierung"""
    # Y-Rotation
    qy = math.radians(rotation_y)
    x_new = x * math.cos(qy) - z * math.sin(qy)
    z_new = x * math.sin(qy) + z * math.cos(qy)
    x, z = x_new, z_new

    # X-Rotation
    qx = math.radians(rotation_x)
    y_new = y * math.cos(qx) - z * math.sin(qx)
    z_new = y * math.sin(qx) + z * math.cos(qx)
    y, z = y_new, z_new

    # Z-Skalierung für bessere Sichtbarkeit
    z = z * z_scale

    # Perspektivische Projektion
    cx, cy = width / 2, height / 2
    screen_x = cx + (x * scale)
    screen_y = cy - (y * scale)

    return screen_x, screen_y, z



--- Processing Audio Data ---


In [ ]:
import os

def create_enhanced_poster(dataframe, youtube_url, output_filename="poster_enhanced.png"):
    print("--- Rendering Photorealistic 3D Poster ---")

    W, H = 4961, 7016  # A2 @ 300 DPI
    bg_color   = (10, 10, 12)
    grid_color = (255, 255, 255)
    axis_color = (255, 255, 255)
    text_color = (255, 255, 255)

    img = Image.new('RGB', (W, H), color=bg_color)
    draw = ImageDraw.Draw(img, 'RGBA')

    scale  = 2250
    rot_y  = 45
    rot_x  = 35
    z_scale = 1.5

    poster_offset_y = 500

    AX_SCALE   = 0.70
    AXIS_EXTRA = 0.0

    try:
        font_h   = ImageFont.truetype("LiberationMono-Bold.ttf", 180)
        font_sub = ImageFont.truetype("LiberationMono-Regular.ttf", 90)
        font_tick = ImageFont.truetype("LiberationMono-Regular.ttf", 60)
    except:
        font_h = font_sub = font_tick = ImageFont.load_default()

    # --- Datenbereich ---
    x_min, x_max = dataframe["x"].min(), dataframe["x"].max()
    y_min, y_max = dataframe["y"].min(), dataframe["y"].max()
    z_min, z_max = dataframe["z_plot"].min(), dataframe["z_plot"].max()

    x0, x1 = x_min, x_max

    pad_y = 0.10 * (y_max - y_min) if y_max > y_min else 0.0
    y0, y1 = y_min - pad_y, y_max

    z0, z1 = z_min, z_max

    cx3d = 0.5 * (x0 + x1)
    cy3d = 0.5 * (y0 + y1)
    cz3d = 0.5 * (z0 + z1)

    def project(p):
        return project_3d_to_2d(
            p[0] - cx3d,
            p[1] - cy3d,
            p[2] - cz3d,
            W, H, scale,
            rot_y, rot_x, z_scale
        )

    def draw_3d_line(p1, p2, width, color, alpha):
        x1p, y1p, z1p = project(p1)
        x2p, y2p, z2p = project(p2)
        y1p += poster_offset_y
        y2p += poster_offset_y
        draw.line((x1p, y1p, x2p, y2p), fill=color + (alpha,), width=width)
        return (x1p, y1p, z1p), (x2p, y2p, z2p)

    def norm_x(x): return (x - x0) / (x1 - x0) if x1 != x0 else 0.0
    def norm_y(y): return (y - y0) / (y1 - y0) if y1 != y0 else 0.0
    def norm_z(z): return (z - z0) / (z1 - z0) if z1 != z0 else 0.0

    # --- GRID + ACHSEN (Hintergrund) ---
    print("Drawing grid and axes...")

    n_grid_z = 9

    x_ticks_world = [x0 + (x1 - x0) * v for v in np.linspace(0, AX_SCALE, 5)]
    y_ticks_world = [y0 + (y1 - y0) * v for v in np.linspace(0, AX_SCALE, 5)]
    z_ticks_world = np.linspace(z0, z1, n_grid_z)

    origin = (x0, y0, z0)

    x_end = (x0 + (x1 - x0) * AX_SCALE, y0, z0)
    y_end = (x0, y0 + (y1 - y0) * AX_SCALE, z0)
    z_end = (x0, y0, z1 + AXIS_EXTRA * (z1 - z0))

    draw_3d_line(origin, x_end, width=20, color=axis_color, alpha=255)
    draw_3d_line(origin, y_end, width=20, color=axis_color, alpha=255)
    draw_3d_line(origin, z_end, width=20, color=axis_color, alpha=255)

    # --- Achsenticks + Labels ---
    print("Adding axis labels and ticks...")

    tick_len_x = 0.05 * (y1 - y0)
    tick_len_y = 0.05 * (x1 - x0)
    tick_len_z = 0.05 * (x1 - x0)

    # X Achse
    for i, xv in enumerate(x_ticks_world):
        if i == 0:
            continue
        p1 = (xv, y0, z0)
        p2 = (xv, y0 - tick_len_x, z0)
        draw_3d_line(p1, p2, width=4, color=axis_color, alpha=200)
        lx, ly, _ = project((xv, y0 - 1.8 * tick_len_x, z0))
        ly += poster_offset_y
        draw.text((lx, ly), f"{norm_x(xv):.1f}",
                  fill=text_color, font=font_tick, anchor="mm")

    lx, ly, _ = project(x_end)
    ly += poster_offset_y
    draw.text((lx - 40, ly - 170), "Time Progression (X)",
              fill=axis_color, font=font_sub, anchor="lm")

    # Y Achse
    for i, yv in enumerate(y_ticks_world):
        if i == 0:
            continue
        p1 = (x0, yv, z0)
        p2 = (x0 - tick_len_y, yv, z0)
        draw_3d_line(p1, p2, width=4, color=axis_color, alpha=200)
        lyx, lyy, _ = project((x0 - 1.8 * tick_len_y, yv, z0))
        lyy += poster_offset_y
        draw.text((lyx, lyy), f"{norm_y(yv):.1f}",
                  fill=text_color, font=font_tick, anchor="mm")

    lyx, lyy, _ = project(y_end)
    lyy += poster_offset_y
    draw.text((lyx, lyy - 180), "Time Progression (Y)",
              fill=axis_color, font=font_sub, anchor="rm")

    # Z Achse
    for i, zv in enumerate(z_ticks_world):
        if i == 0:
            continue
        p1 = (x0, y0, zv)
        p2 = (x0 - tick_len_z, y0, zv)
        draw_3d_line(p1, p2, width=4, color=axis_color, alpha=200)
        lzx, lzy, _ = project((x0 - 1.8 * tick_len_z, y0, zv))
        lzy += poster_offset_y
        draw.text((lzx + 10, lzy), f"{norm_z(zv):.1f}",
                  fill=text_color, font=font_tick, anchor="rm")

    lzx, lzy, _ = project(z_end)
    lzy += poster_offset_y
    draw.text((lzx, lzy + 150), "LOUDNESS (RMS)",
              fill=axis_color, font=font_sub, anchor="mm")

    # --- PHOTOREALISTIC RENDERING WITH NEON GLOW ---
    print("Rendering photorealistic data points with neon glow...")

    # Create multiple layers for depth and lighting effects
    neon_glow_layer = Image.new('RGBA', (W // 3, H // 3), (0, 0, 0, 0))
    draw_glow = ImageDraw.Draw(neon_glow_layer)
    
    # Shadow layer for depth
    shadow_layer = Image.new('RGBA', (W, H), (0, 0, 0, 0))
    draw_shadow = ImageDraw.Draw(shadow_layer)

    nodes = []
    for _, row in dataframe.iterrows():
        sx, sy, sz = project((row["x"], row["y"], row["z_plot"]))
        sy += poster_offset_y
        nodes.append((sx, sy, sz, row["color"]))
    nodes.sort(key=lambda k: k[2])

    # Define light source position (top-right-front)
    light_pos = np.array([W * 0.7, H * 0.3, 100])
    
    # Enhanced neon glow with volumetric effect
    scale_glow = 1.0 / 3.0
    for sx, sy, sz, col in nodes:
        fx, fy = sx * scale_glow, sy * scale_glow
        # Depth-based glow intensity
        depth_factor = (sz - min([n[2] for n in nodes])) / (max([n[2] for n in nodes]) - min([n[2] for n in nodes]) + 0.001)
        glow_alpha = int(180 * (1 - depth_factor * 0.3))
        
        # Boost neon colors for vibrant glow
        neon_col = (
            min(255, int(col[0] * 1.3)),
            min(255, int(col[1] * 1.3)),
            min(255, int(col[2] * 1.3))
        )
        
        # Multiple glow rings for neon effect
        for glow_ring in range(3):
            gr = (25 + glow_ring * 15) * 3
            ring_alpha = int(glow_alpha * (1 - glow_ring * 0.25))
            draw_glow.ellipse(
                [fx - gr, fy - gr, fx + gr, fy + gr],
                fill=(neon_col[0], neon_col[1], neon_col[2], ring_alpha)
            )
    
    # Apply blur for neon glow diffusion
    neon_glow_layer = neon_glow_layer.filter(ImageFilter.GaussianBlur(35))
    neon_glow_layer = neon_glow_layer.resize((W, H), resample=Image.BICUBIC)
    img.paste(neon_glow_layer, (0, 0), neon_glow_layer)

    # Draw subtle shadows first (below points)
    for sx, sy, sz, col in nodes:
        # Calculate shadow offset based on light position
        light_vec = light_pos - np.array([sx, sy, sz])
        light_dist = np.linalg.norm(light_vec)
        shadow_offset_x = -12 * (light_pos[0] - sx) / W
        shadow_offset_y = 20 * (light_pos[1] - sy) / H
        
        shadow_x = sx + shadow_offset_x
        shadow_y = sy + shadow_offset_y
        shadow_size = 18
        
        # Depth-based shadow opacity (lighter for neon aesthetic)
        depth_factor = (sz - min([n[2] for n in nodes])) / (max([n[2] for n in nodes]) - min([n[2] for n in nodes]) + 0.001)
        shadow_alpha = int(60 * (1 - depth_factor * 0.6))
        
        draw_shadow.ellipse(
            [shadow_x - shadow_size, shadow_y - shadow_size, 
             shadow_x + shadow_size, shadow_y + shadow_size],
            fill=(0, 0, 0, shadow_alpha)
        )
    
    # Blur shadows for realism
    shadow_layer = shadow_layer.filter(ImageFilter.GaussianBlur(12))
    img.paste(shadow_layer, (0, 0), shadow_layer)

    # Draw photorealistic neon points with enhanced colors
    draw_final = ImageDraw.Draw(img)
    for sx, sy, sz, col in nodes:
        # Calculate lighting based on distance to light source
        light_vec = light_pos - np.array([sx, sy, sz])
        light_dist = np.linalg.norm(light_vec)
        # Higher base intensity to keep neon colors vibrant
        light_intensity = max(0.7, min(1.2, 600 / (light_dist + 80)))
        
        # Boost neon colors - keep them vibrant and saturated
        neon_boost = 1.15
        lit_col = (
            min(255, int(col[0] * light_intensity * neon_boost)),
            min(255, int(col[1] * light_intensity * neon_boost)),
            min(255, int(col[2] * light_intensity * neon_boost))
        )
        
        # Size varies with depth for depth of field effect
        depth_factor = (sz - min([n[2] for n in nodes])) / (max([n[2] for n in nodes]) - min([n[2] for n in nodes]) + 0.001)
        pt_size = 18 + int(6 * depth_factor)
        
        # Draw base sphere with neon gradient effect (brighter core)
        for r_offset in range(pt_size, 0, -2):
            alpha_gradient = int(255 * (r_offset / pt_size))
            # Brighter gradient for neon look
            brightness = 0.5 + 0.5 * (r_offset / pt_size)
            grad_col = (
                min(255, int(lit_col[0] * brightness)),
                min(255, int(lit_col[1] * brightness)),
                min(255, int(lit_col[2] * brightness))
            )
            draw_final.ellipse(
                [sx - r_offset, sy - r_offset, sx + r_offset, sy + r_offset],
                fill=grad_col + (alpha_gradient,)
            )
        
        # Bright neon core with color tint
        core_size = max(8, int(pt_size * 0.4))
        core_col = (
            min(255, int((lit_col[0] + 255) * 0.7)),
            min(255, int((lit_col[1] + 255) * 0.7)),
            min(255, int((lit_col[2] + 255) * 0.7))
        )
        draw_final.ellipse(
            [sx - core_size, sy - core_size, sx + core_size, sy + core_size],
            fill=core_col
        )
        
        # Ultra-bright center hotspot for neon glow
        hotspot_size = max(3, int(core_size * 0.5))
        draw_final.ellipse(
            [sx - hotspot_size, sy - hotspot_size, 
             sx + hotspot_size, sy + hotspot_size],
            fill=(255, 255, 255, 255)
        )

    # --- Titel + QR ---
    print("Adding title and QR code...")

    draw_final.text((150, 150), "3D SONIC PHYLLOTAXIS",
                    fill=axis_color, font=font_h)
    draw_final.text((150, 380), "LOUDNESS MAP (RMS)",
                    fill=text_color, font=font_sub)

    qr = qrcode.QRCode(box_size=15, border=2)
    qr.add_data(youtube_url)
    qr.make(fit=True)
    qr_img = qr.make_image(fill_color="black",
                           back_color="white").convert("RGB")
    img.paste(qr_img, (W - qr_img.width - 150, H - qr_img.height - 150))

    # --- EXPORTS ---
    base, _ = os.path.splitext(output_filename)
    png_path  = f"{base}.png"
    tiff_path = f"{base}_uncompressed.tiff"

    img.save(png_path, dpi=(300, 300))
    img.save(tiff_path, compression="none", dpi=(300, 300))

    print(f"✓ Saved PNG:  {png_path}")
    print(f"✓ Saved TIFF: {tiff_path}")
    return tiff_path


In [ ]:
final_file = create_enhanced_poster(df_final, "https://www.youtube.com/watch?v=xwTPvcPYaOo&list=RDxwTPvcPYaOo&start_radio=1")
from google.colab import files
files.download(final_file)   # lädt das TIFF


--- Rendering Enhanced 3D Poster ---
Drawing grid and axes...
Adding axis labels and ticks...
Rendering data points...
Adding title and QR code...
✓ Saved PNG:  poster_enhanced.png
✓ Saved TIFF: poster_enhanced_uncompressed.tiff


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>